#### Partie 2 : Modélisation et Prédiction avec XGBoost

Après avoir réalisé la régression linéaire et Lasso, cette partie du notebook présente une approche avancée de modélisation pour la prédiction du prix des voitures à l’aide de XGBoost, un algorithme de boosting performant pour les tâches de régression.

---

**1. Importation des bibliothèques**  
Les bibliothèques nécessaires pour la manipulation des données, le prétraitement, la modélisation et l’évaluation (pandas, numpy, xgboost, scikit-learn, joblib) sont importées.

**2. Chargement et exploration des données**  
Le jeu de données nettoyé est chargé, puis examiné pour vérifier sa structure, ses dimensions et l’absence de valeurs manquantes.

**3. Gestion des valeurs aberrantes**  
Les valeurs extrêmes du prix sont identifiées et supprimées à l’aide de la méthode de l’écart interquartile (IQR), afin d’améliorer la robustesse du modèle.

**4. Prétraitement des données**  
- Sélection des caractéristiques pertinentes pour la prédiction.
- Encodage des variables catégorielles avec LabelEncoder, car XGBoost ne gère pas directement les textes.
- Mise à l’échelle des variables numériques avec StandardScaler pour homogénéiser les échelles.

**5. Division du jeu de données**  
Les données sont séparées en ensembles d’entraînement (80%) et de test (20%) pour évaluer la performance du modèle sur des données non vues.

**6. Entraînement du modèle XGBoost**  
Un modèle XGBoostRegressor est entraîné sur les données prétraitées.

**7. Prédiction et évaluation**  
- Prédiction des prix sur l’ensemble de test.
- Calcul des métriques d’évaluation : MAE, RMSE et R² pour mesurer la qualité des prédictions.
- Visualisation de l’importance des caractéristiques pour comprendre les variables les plus influentes.
- Comparaison des valeurs réelles et prédites sur un échantillon.

**8. Prédiction sur une nouvelle voiture**  
Le notebook montre comment préparer les données d’un nouveau véhicule, appliquer les mêmes transformations, puis prédire son prix avec le modèle entraîné.

**9. Sauvegarde du modèle et des objets de prétraitement**  
Le modèle, le scaler, les encoders et la liste des colonnes sont sauvegardés pour une utilisation future sans avoir à refaire tout le pipeline.

---

Cette démarche permet d’obtenir un modèle robuste, réutilisable et performant pour la prédiction du prix des voitures d’occasion à partir de leurs caractéristiques.

**Importer les bibliothèques nécessaires**

In [91]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

**Chargement des données**

In [92]:
import os

# Lecture du fichier nettoyé
# Définir le chemin relatif à la racine du projet
project_root = os.path.dirname(os.path.abspath(''))
data_path = os.path.join(project_root, 'data', 'avito_cars_clean.csv')

df = pd.read_csv(data_path)

# Afficher les cinq premières lignes des données
df.head()

fuel_options = df['carburant'].unique()
print("Available fuel options:", fuel_options)

Available fuel options: ['Essence' 'Diesel' 'Hybride' 'Electrique' 'LPG']


**Examen des données**

In [93]:
print(f"Dimensions des données : {df.shape}")
print(f"Statistiques des prix : Min={df['prix'].min()}, Max={df['prix'].max()}, Moyenne={df['prix'].mean()}")
print(f"Colonnes : {df.columns.tolist()}")

Dimensions des données : (68415, 11)
Statistiques des prix : Min=5200, Max=1000000, Moyenne=138682.62904333844
Colonnes : ['annee', 'boite', 'carburant', 'kilometrage', 'marque', 'modele', 'nombre_portres', 'premiere_main', 'puissance_fiscale', 'etat', 'prix']


**Vérifier et nettoyer les données**

In [94]:
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68415 entries, 0 to 68414
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   annee              68415 non-null  int64  
 1   boite              68415 non-null  object 
 2   carburant          68415 non-null  object 
 3   kilometrage        68415 non-null  float64
 4   marque             68415 non-null  object 
 5   modele             68415 non-null  object 
 6   nombre_portres     68415 non-null  int64  
 7   premiere_main      68415 non-null  object 
 8   puissance_fiscale  68415 non-null  int64  
 9   etat               68415 non-null  object 
 10  prix               68415 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 5.7+ MB
None
annee                0
boite                0
carburant            0
kilometrage          0
marque               0
modele               0
nombre_portres       0
premiere_main        0
puissance_fiscal

***Cela signifie que les données sont bien structurées et prêtes pour l'analyse, et qu’il n’y a pas de données manquantes à traiter.***

**Gestion des valeurs aberrantes (AVANT la division des données)**

In [95]:
Q1 = df['prix'].quantile(0.25)
Q3 = df['prix'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
    
print(f"Filtrage des prix entre {lower_bound} et {upper_bound}")
df_filtered = df[(df['prix'] >= lower_bound) & (df['prix'] <= upper_bound)]
print(f"Valeurs aberrantes supprimées : {df.shape[0] - df_filtered.shape[0]}")

Filtrage des prix entre -95505.0 et 332503.0
Valeurs aberrantes supprimées : 4041


**Sélection des caractéristiques et prétraitement**

In [96]:
# Garder toutes les caractéristiques utilisées dans Lasso/LR
features = ['annee', 'kilometrage', 'puissance_fiscale', 'etat', 'marque', 'modele', 'boite', 'carburant', 'nombre_portres', 'premiere_main']
target = 'prix'

# Création de X et y
X = df_filtered[features]
y = df_filtered[target]

***XGBoost ne gère pas les textes directement, donc on convertit les colonnes textuelles en chiffres.***

In [102]:
# Encodage des variables catégorielles
categorical_cols = ['etat', 'marque', 'modele', 'boite', 'carburant', 'premiere_main']
label_encoders = {}
for col in categorical_cols:
    if col in X.columns:
        le = LabelEncoder()
        X.loc[:, col] = le.fit_transform(X[col])
        label_encoders[col] = le

# Print the encoders for debugging
for col, encoder in label_encoders.items():
    print(f"Encoded {col} with {len(encoder.classes_)} unique values")

Encoded etat with 7 unique values
Encoded marque with 70 unique values
Encoded modele with 770 unique values
Encoded boite with 2 unique values
Encoded carburant with 5 unique values
Encoded premiere_main with 2 unique values


**Suppression des lignes avec des valeurs manquantes**

In [103]:
X = X.dropna()
y = y[X.index]

**On va entraîner le modèle sur 80% des données et tester sur 20%.**

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Mise à l'échelle des caractéristiques**

In [105]:
scaler = StandardScaler()
numeric_cols = ['annee', 'kilometrage', 'puissance_fiscale', 'nombre_portres']
X_train_numeric = scaler.fit_transform(X_train[numeric_cols])
X_test_numeric = scaler.transform(X_test[numeric_cols])
    
# Conversion en DataFrame pour faciliter la manipulation
X_train_numeric_df = pd.DataFrame(X_train_numeric, columns=numeric_cols, index=X_train.index)
X_test_numeric_df = pd.DataFrame(X_test_numeric, columns=numeric_cols, index=X_test.index)
    
# Remplacement des colonnes numériques par leurs versions mises à l'échelle
for col in numeric_cols:
    X_train[col] = X_train_numeric_df[col]
    X_test[col] = X_test_numeric_df[col]

**Création et entraînement du modèle XGBoost**

In [106]:
# Assurez-vous que toutes les colonnes catégorielles sont bien de type int
for col in categorical_cols:
    if col in X_train.columns:
        X_train[col] = X_train[col].astype(int)
        X_test[col] = X_test[col].astype(int)

model = xgb.XGBRegressor(
    objective='reg:squarederror', 
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)

model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

**Prédictions avec les données de test**

In [107]:
y_pred = model.predict(X_test)

**Évaluation du modèle**

In [108]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
    
print("\nPerformance du modèle XGBoost :")
print(f"Erreur absolue moyenne (MAE) : {mae:.2f}")
print(f"Erreur quadratique moyenne (RMSE) : {rmse:.2f}")
print(f"R² (coefficient de détermination) : {r2:.4f}")


Performance du modèle XGBoost :
Erreur absolue moyenne (MAE) : 16616.11
Erreur quadratique moyenne (RMSE) : 25671.76
R² (coefficient de détermination) : 0.8703


**Visualisation de l'importance des caractéristiques**

In [109]:
importance = model.feature_importances_
feature_names = X_train.columns
    
# Créer un DataFrame pour l'importance des caractéristiques
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)
    
print("\nImportance des caractéristiques :")
print(feature_importance)


Importance des caractéristiques :
             Feature  Importance
6              boite    0.433601
0              annee    0.338555
7          carburant    0.057080
2  puissance_fiscale    0.048902
4             marque    0.032758
9      premiere_main    0.031120
5             modele    0.023275
3               etat    0.018635
8     nombre_portres    0.008430
1        kilometrage    0.007644


**Comparaison des valeurs prédites vs réelles (échantillon)**

In [110]:
comparison = pd.DataFrame({
    'Réel': y_test.values[:20],
    'Prédit': y_pred[:20],
    'Différence': y_test.values[:20] - y_pred[:20]
})
print("\nComparaison des prédictions (20 premiers exemples) :")
print(comparison)


Comparaison des prédictions (20 premiers exemples) :
      Réel         Prédit    Différence
0    65000   68464.453125  -3464.453125
1    58000   69980.171875 -11980.171875
2   165000  156716.390625   8283.609375
3   250000  275530.718750 -25530.718750
4    65000   54984.636719  10015.363281
5    27000   27971.070312   -971.070312
6    50000   50752.527344   -752.527344
7   148000  146226.609375   1773.390625
8   217000  178651.484375  38348.515625
9    85000   89047.109375  -4047.109375
10   45000   44099.964844    900.035156
11  330000  261626.984375  68373.015625
12  135000  117109.640625  17890.359375
13  120000  122523.757812  -2523.757812
14  180000  165431.953125  14568.046875
15   52000   51863.378906    136.621094
16  105000  103265.312500   1734.687500
17  290000  274928.937500  15071.062500
18   90000   98227.492188  -8227.492188
19   83000   80396.601562   2603.398438


**Prédiction pour une nouvelle voiture**

In [111]:
# Predict price for new car: Volkswagen Tiguan, 2009, 200000 km, 8 CV, Automatique, Diesel, Excellent
new_car = pd.DataFrame({
    'annee': [2009],
    'kilometrage': [200000],
    'puissance_fiscale': [8],
    'etat': ['Excellent'],
    'marque': ['Volkswagen'],
    'modele': ['Tiguan'],
    'boite': ['Automatique'],
    'carburant': ['Diesel'],
    'nombre_portres': [5],
    'premiere_main': ['Non']
})

# Apply label encoding to categorical columns
categorical_cols = ['etat', 'marque', 'modele', 'boite', 'carburant', 'premiere_main']
for col in categorical_cols:
    if col in new_car.columns:
        le = label_encoders[col]
        # Handle unseen labels by assigning a default value (e.g., mode of encoded values)
        new_car[col] = new_car[col].apply(lambda x: x if x in le.classes_ else le.classes_[0])
        new_car[col] = le.transform(new_car[col])

# Scale numerical features
numeric_cols = ['annee', 'kilometrage', 'puissance_fiscale', 'nombre_portres']
new_car_numeric = scaler.transform(new_car[numeric_cols])
new_car_numeric_df = pd.DataFrame(new_car_numeric, columns=numeric_cols, index=new_car.index)

# Replace numerical columns with scaled values
for col in numeric_cols:
    new_car[col] = new_car_numeric_df[col]

# Ensure the new car has the same columns as training data
new_car = new_car[X.columns]

# Make prediction
xgb_pred = model.predict(new_car)

# Print predicted price in MAD
print("\nPredicted Price for Volkswagen Tiguan (2009, 200000 km, 8 CV, Automatique, Diesel, Excellent):")
print(f"XGBoost: {xgb_pred[0]:.2f} MAD")


Predicted Price for Volkswagen Tiguan (2009, 200000 km, 8 CV, Automatique, Diesel, Excellent):
XGBoost: 92567.20 MAD


**Sauvegarde du modèle et des objets nécessaires**

In [112]:
# Sauvegarder le scaler
# Définir le chemin du dossier pkl à la racine du projet
pkl_dir = os.path.join(project_root, 'pkl-files')
os.makedirs(pkl_dir, exist_ok=True)

# Sauvegarder le scaler
joblib.dump(scaler, os.path.join(pkl_dir, 'xgboost_scaler.pkl'))

# Sauvegarder le modèle
joblib.dump(model, os.path.join(pkl_dir, 'xgboost_model.pkl'))

# Sauvegarder les colonnes utilisées
joblib.dump(X.columns, os.path.join(pkl_dir, 'xgboost_columns.pkl'))

# Sauvegarder les label encoders
joblib.dump(label_encoders, os.path.join(pkl_dir, 'xgboost_label_encoders.pkl'))

print("Le scaler, le modèle, les colonnes et les label encoders ont été sauvegardés sous forme de fichiers .pkl.")

Le scaler, le modèle, les colonnes et les label encoders ont été sauvegardés sous forme de fichiers .pkl.


**Travail Réalisé par EL ANNASI Nada et EL-GHEFYRY Salma**